In [ ]:
!pip install langchain
!pip install langgraph
!pip install -qU langchain[google-genai]
!pip install allosaurus
!pip install gTTS
!pip install pydub

In [6]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-pro", model_provider="google_genai")

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
chat_prompt_template = ChatPromptTemplate.from_messages(
    [("system", "Act as an expert Chinese language tutor with over 10 years of experience. \
    Use the following communication style: encouraging, patient, culturally sensitive and systematically progressive. \开始
    Gently correct mistakes (including pronounciation mistakes) in real time. \
    Regularly highlight student achievements and improvements to maintain motivation. \
    You are tutoring a native US English speaker. \
    Format your responses so that they are concise, teaching a little bit at a time \
    Format your responses so that a text-to-speech system that can only pronounce chinese character and english character\
    ,not pinyin, can use them. \
    Don't use parenthetical phrases\
    When speaking Chinese, never use vocabulary about the pre-2021 HSK-3 level under any circumstances.",), \
     MessagesPlaceholder(variable_name="messages"),]
)

In [12]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    prompt = chat_prompt_template.invoke(state["messages"])
    response = model.invoke(prompt)
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

#Adding Memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
from langchain_core.messages import HumanMessage
from gtts import gTTS
from io import BytesIO
from pydub import AudioSegment
from pydub.playback import play

config = {"configurable": {"thread_id": "CM"}}
while True:
  user_input = input("You>:")
  input_messages = [HumanMessage(user_input)]
  output = app.invoke({"messages": input_messages}, config)
  last_message = output["messages"][-1]
  print("Teacher>:", end="")
  last_message.pretty_print()
  # Use gTTS and pygame to say the AI message with Taiwanese voice
  mp3_file_like = BytesIO()
  tts = gTTS(text=last_message.text(), lang='zh-TW', slow=False)
  tts.write_to_fp(mp3_file_like)
  mp3_file_like.seek(0)
  # Convert the file-like object to an AudioSegment
  audio = AudioSegment.from_mp3(BytesIO(mp3_file_like.read()))
  # Play the sound
  play(audio)
  mp3_file_like.close()